In [93]:
import re
import fitz

from langchain_cohere import ChatCohere
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [114]:
llm = ChatCohere(
    model="command-r-plus",
    temperature=0.0,
)

In [164]:
prompt_template = """
Write a summary of the following in 900 words.
Don't include any personal opinions or interpretations.
You are helping in writing summary of a wider text, so write as if there summaries behind and after this one, but don't include any information that isn't in the text.
DO NOT INTRODUCE ANY CHARACTER.

Text:
{chapters}
"""

prompt = PromptTemplate.from_template(prompt_template)

In [165]:
summary_chain = (
    prompt
    | llm
    | StrOutputParser()
)

In [51]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [52]:
book_text = extract_text_from_pdf(pdf_path="crime-and-punishment.pdf")

In [66]:
chapters = re.split(r'Chapter [IVXLCDM]+', book_text)

In [79]:
chapters[39] = chapters[39].split("Epilogue")[0]

In [80]:
chapters[39]

'\nW\nhen he went into Sonia’s room, it was already getting \ndark. All day Sonia had been waiting for him in ter-\nrible anxiety. Dounia had been waiting with her. She had \ncome to her that morning, remembering Svidrigaïlov’s \nwords that Sonia knew. We will not describe the conver-\nsation and tears of the two girls, and how friendly they \nbecame. Dounia gained one comfort at least from that in-\nterview, that her brother would not be alone. He had gone \nto her, Sonia, first with his confession; he had gone to her \nfor human fellowship when he needed it; she would go with \nhim wherever fate might send him. Dounia did not ask, but \nshe knew it was so. She looked at Sonia almost with rever-\nence and at first almost embarrassed her by it. Sonia was \nalmost on the point of tears. She felt herself, on the contrary, \nhardly worthy to look at Dounia. Dounia’s gracious image \nwhen she had bowed to her so attentively and respectfully at \ntheir first meeting in Raskolnikov’s room ha

In [81]:
chapters = chapters[1:]

In [82]:
chapters[0]

'\nO\nn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto-\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at-\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been in an over-\nstrained irritable condition, verging on hypochondria. He \nhad become so completely absorbed in himself, and iso-\nlated 

In [98]:
def group_chapters(chapters, group_size=2):
    return [chapters[i:i+group_size] for i in range(0, len(chapters), group_size)]

In [99]:
grouped_chapters = group_chapters(chapters)

In [100]:
grouped_chapters[0]

['\nO\nn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto-\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at-\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been in an over-\nstrained irritable condition, verging on hypochondria. He \nhad become so completely absorbed in himself, and iso-\nlated

In [101]:
grouped_chapters[-1]

['\nW\nhen he went into Sonia’s room, it was already getting \ndark. All day Sonia had been waiting for him in ter-\nrible anxiety. Dounia had been waiting with her. She had \ncome to her that morning, remembering Svidrigaïlov’s \nwords that Sonia knew. We will not describe the conver-\nsation and tears of the two girls, and how friendly they \nbecame. Dounia gained one comfort at least from that in-\nterview, that her brother would not be alone. He had gone \nto her, Sonia, first with his confession; he had gone to her \nfor human fellowship when he needed it; she would go with \nhim wherever fate might send him. Dounia did not ask, but \nshe knew it was so. She looked at Sonia almost with rever-\nence and at first almost embarrassed her by it. Sonia was \nalmost on the point of tears. She felt herself, on the contrary, \nhardly worthy to look at Dounia. Dounia’s gracious image \nwhen she had bowed to her so attentively and respectfully at \ntheir first meeting in Raskolnikov’s room h

In [153]:
summary_chain.invoke(
    {
        "chapters": grouped_chapters[0]
    }
)

'On a hot July evening, a young man named Raskolnikov leaves his lodging and walks towards a bridge in Petersburg, Russia. He is anxious about encountering his landlady, whom he owes money, and prefers to sneak in and out of the building unnoticed. As he walks, he reflects on his plans for a potential crime and his fear of taking action. He also notes the oppressive heat, the dirty surroundings, and the drunk men he passes by. Raskolnikov enters a tavern and sits down, ordering a beer. He observes the other patrons, including a drunk man singing a song and a group of men and a girl with a concertina. He is particularly drawn to a man named Marmeladov, a former government clerk who appears unkempt and disheveled. Marmeladov approaches Raskolnikov and strikes up a conversation, sharing his life story and expressing his shame and guilt over his alcoholism and inability to provide for his family. Marmeladov describes his wife, Katerina Ivanovna, and their three children, as well as his ste

In [166]:
summaries = []

for chapters in grouped_chapters:
    summary = summary_chain.invoke(
        {
            "chapters": chapters
        }
    )
    
    summaries.append(summary)

In [167]:
summaries

['On a hot July evening, a young man named Raskolnikov leaves his lodging and walks towards a bridge in Petersburg, Russia. He is anxious about encountering his landlady, as he owes her money and has been avoiding her. Raskolnikov is in a state of mental distress, feeling isolated and overwhelmed by his poverty. He contemplates taking a "new step" that seems to involve some kind of criminal act, but he is hesitant and filled with self-doubt. As he walks, he observes the squalor and drunkenness around him, which further distresses him. He reaches the apartment of an elderly pawn broker, Alyona Ivanovna, from whom he has previously pawned items. He plans to pawn a watch, but the transaction does not go as he expected, and he leaves feeling frustrated and ashamed. \n\nRaskolnikov enters a tavern, where he encounters a drunk man named Marmeladov, who starts a long, rambling monologue. Marmeladov reveals details about his life, including his alcoholism, his unemployed status, and his family

In [161]:
def write_summary_to_file(summaries, output_file):
    with open(output_file, "w") as f:
        for i, summary in enumerate(summaries):
            f.write(summary)
            f.write("\n\n")

In [168]:
write_summary_to_file(summaries, "crime-and-punishment-summary.txt")